In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style



In [2]:
def read_dataset(path, name):
    df = pd.read_csv(path + name, header=None)
    return df

In [3]:
def initialize_membership_matrix(df, k):
    membership_matrix = np.random.dirichlet(np.ones(k),size=len(df))
    df_membership_matrix = pd.DataFrame(membership_matrix)
    return df_membership_matrix

In [4]:
def find_centroids(df, k, membership_matrix, m):
    centroids = np.zeros((k, len(df.columns)))

    for i in range(k):
        sum_membership = sum(np.power(membership_matrix[i], m))
        for j in range (len(df.columns)):
            c = sum(np.multiply(np.power(membership_matrix[i], m), df[j]))/sum_membership
            centroids[i, j] = c

    return centroids    

In [5]:
def calculate_membership_matrix(df, centroids):
    m = 2
    EuclidianDistance = pd.DataFrame()
    inverse_EuclidianDistance = pd.DataFrame()
    membership_matrix = pd.DataFrame()
    for i in range(0, len(centroids)):
        x = 0
        for j in range(0, len(df.columns)):
            x = np.add(x, np.power((df.iloc[ : , j] - centroids[i,j]), 2))
        x = np.sqrt(x)
        EuclidianDistance[len(EuclidianDistance.columns)] = x

    inverse_EuclidianDistance = np.power((1 / EuclidianDistance), 1/(m-1))
    sum = inverse_EuclidianDistance.sum(axis=1)

    for i in range(k):
        membership_matrix[len(membership_matrix.columns)] = 1 / np.power(EuclidianDistance[i], 1/(m-1))  / sum

    return membership_matrix   

In [6]:
def c_means(df, k, max_it):
    #initialing weights randomly
    membership_matrix = initialize_membership_matrix(df, k)
    centroids = np.zeros((k, len(df.columns)))
    for i in range(max_it):
        temp_centroids = find_centroids(df, k, membership_matrix, 2)
        if centroids.all == temp_centroids.all:
            print("clustering finished, it", i)
            return
        centroids = temp_centroids
        membership_matrix = calculate_membership_matrix(df, centroids)

    return membership_matrix

In [7]:
df = read_dataset("datasets/", "data2.csv")
df

,0,1,2,3
0,0.432610,0.382513,0.417406,0.381025
1,0.804723,0.846790,0.827204,0.841531
2,0.399435,0.445441,0.377028,0.407129
3,0.586848,0.605577,0.664818,0.654835
4,0.745649,0.727290,0.773916,0.754974
...,...,...,...,...
1207,0.105985,0.183894,0.190385,0.137578
1208,0.660885,0.599424,0.678428,0.630584
1209,0.023225,0.045101,0.101654,0.044252
1210,0.187667,0.130879,0.143621,0.177495


In [8]:
k = 5
max_it = 500
c_means(df, k, max_it)

UnboundLocalError: local variable 'centroids' referenced before assignment